In [27]:
import requests
from google.transit import gtfs_realtime_pb2
import json

api_url = "https://realtime.hsl.fi/realtime/vehicle-positions/v2/hsl"

# Fetch the GTFS-RT payload
response = requests.get(api_url)
payload = response.content

# Create a GTFS-RT feed message
feed = gtfs_realtime_pb2.FeedMessage()

# Parse and decode the payload
feed.ParseFromString(payload)

# Create a list to hold the decoded entities
decoded_entities = []

# Iterate over the feed entities
for entity in feed.entity:
    # Convert the entity to a dictionary
    entity_dict = {
        'id': entity.id,
        'vehicle': {
            'trip': {
                'route_id': entity.vehicle.trip.route_id,
                'start_time': entity.vehicle.trip.start_time,
                'start_date': entity.vehicle.trip.start_date
                # Add other trip attributes as needed
            },
            'position': {
                'latitude': entity.vehicle.position.latitude,
                'longitude': entity.vehicle.position.longitude,
                'bearing': entity.vehicle.position.bearing
                # Add other trip attributes as needed
            },
            'position': {
                'latitude': entity.vehicle.position.latitude,
                'longitude': entity.vehicle.position.longitude,
                'bearing': entity.vehicle.position.bearing,
                'odometer': entity.vehicle.position.odometer,
                'speed' : entity.vehicle.position.speed
                # Add other trip attributes as needed
            },
            'stop_id': entity.vehicle.stop_id,
            'current_status': entity.vehicle.current_status,
            'timestamp': entity.vehicle.timestamp
        }
    }


    # Append the entity dictionary to the list
    decoded_entities.append(entity_dict)

# Convert the list of decoded entities to a JSON array
json_array = json.dumps(decoded_entities, indent=2)

# Print the JSON array
print(json_array)

[
  {
    "id": "vehicle_position_22/943",
    "vehicle": {
      "trip": {
        "route_id": "2510",
        "start_time": "11:35:00",
        "start_date": "20240109"
      },
      "position": {
        "latitude": 60.19532775878906,
        "longitude": 24.890108108520508,
        "bearing": 306.0,
        "odometer": 9663.0,
        "speed": 12.65999984741211
      },
      "stop_id": "1301212",
      "current_status": 2,
      "timestamp": 1704794773
    }
  },
  {
    "id": "vehicle_position_18/166",
    "vehicle": {
      "trip": {
        "route_id": "9841",
        "start_time": "12:05:00",
        "start_date": "20240109"
      },
      "position": {
        "latitude": 60.37295150756836,
        "longitude": 25.264453887939453,
        "bearing": 227.0,
        "odometer": 392.0,
        "speed": 0.10000000149011612
      },
      "stop_id": "9222266",
      "current_status": 1,
      "timestamp": 1704794773
    }
  },
  {
    "id": "vehicle_position_18/167",
    "vehicle